In [1]:
import sys
sys.path.append('/home/hwang/anaconda3/lib/python3.7/site-packages/')

from sklearn import datasets
from sklearn.model_selection import train_test_split

import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import SimpleITK as sitk
import nibabel
import math
import texture

from keras.utils.np_utils import to_categorical
from keras import optimizers 
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, Dropout, Input, Lambda, Reshape, AveragePooling2D, BatchNormalization
from keras.layers.merge import add, concatenate
from keras.applications.resnet50 import ResNet50, decode_predictions

from keras import backend as K
from keras.utils import plot_model

from keras.callbacks import EarlyStopping, LambdaCallback
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization

os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"

# import pcanet as net 

/Users/hwang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hwang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hwang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hwang/opt/anaconda3/lib/python3.7/site-packages/tensorflow/pyth

In [7]:
# delete .DS_Store file in each folder 

# image

imgpath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/T1_norm_niigz/'
# imgpath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/T1/'
imgfile = os.listdir(imgpath)
imgfile.sort()
num_img = len(imgfile)
for item_img in os.listdir(imgpath):
    current_img = os.path.basename(item_img)
    if current_img == '.DS_Store':
        fullpath = os.path.join(imgpath,current_img)
        os.remove(fullpath)
                    
# mask
            
maskpath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/roi_niigz/'
# maskpath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/ROI/'
maskfile = os.listdir(maskpath)
maskfile.sort()
num_mask = len(maskfile)
count_mask = 0
for item_mask in os.listdir(maskpath):
    current_mask = os.path.basename(item_mask)
    if current_mask == '.DS_Store':
        fullpath = os.path.join(maskpath,current_mask)
        os.remove(fullpath)   

In [ ]:
# image segment 

segpath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/segment_niigz/'
segpath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/segment_niigz/'
num_seg = len(segfile)

num = 0

for item_mask in os.listdir(maskpath):

    i = sitk.ReadImage(os.path.join(imgpath,imgfile[num]))
    j = sitk.ReadImage(os.path.join(maskpath,maskfile[num]))
    
    seg = i*j
    segfile = os.path.basename(imgfile[num])
    
    sitk.WriteImage(seg, os.path.join(segpath,segfile))
    
    # target 
    
    if segfile[2] == 'M':
        target[num] = 1
    else:
        target[num] = 0
    
    if num < num_seg:
        num = num + 1

In [5]:
# delete .DS_Store file in each folder 

# seg
            
segpath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_org_niigz/'
# segpath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/segment_niigz/'
segfile = os.listdir(segpath)
segfile.sort()
num_seg = len(segfile)
count_seg = 0
for item_seg in os.listdir(segpath):
    current_seg = os.path.basename(item_seg)
    if current_seg == '.DS_Store':
        fullpath = os.path.join(segpath,current_seg)
        os.remove(fullpath)   

In [ ]:
# delete .DS_Store file in each folder 

# resample
            
resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_niigz/'
resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_org_niigz/'

resampath  = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_niigz/'
resampath  = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_org_niigz/'

resampfile = os.listdir(resampath)
resampfile.sort()
num_seg = len(resampfile)
count_seg = 0
for item_resamp in os.listdir(resampath):
    current_resamp = os.path.basename(item_resamp)
    if current_resamp == '.DS_Store':
        fullpath = os.path.join(resampath,current_resamp)
        os.remove(fullpath)  

In [ ]:
# resample segmented image 

resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_niigz/'
resampath  = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_niigz/'
resampfile = os.listdir(resampath)

i = sitk.ReadImage(os.path.join(segpath,segfile[0]))
sitk.WriteImage(i, os.path.join(resampath,segfile[0]))
num_seg = len(segfile)

num = 1

for item_seg in os.listdir(segpath):
   
    j = sitk.ReadImage(os.path.join(segpath,segfile[num]))
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(i)
    resampled = resampler.Execute(j)
    resampfile = os.path.basename(segfile[num])
    
    sitk.WriteImage(resampled, os.path.join(resampath,resampfile))
    
    if num < num_seg-1:
        num = num + 1

In [9]:
# resample original image (T1norm) 

resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_org_niigz/'
# resampath  = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_org_niigz/'
resampfile = os.listdir(resampath)

i = sitk.ReadImage(os.path.join(imgpath,imgfile[0]))
sitk.WriteImage(i, os.path.join(resampath,imgfile[0]))
num_img = len(imgfile)

num = 1

for item_seg in os.listdir(imgpath):
   
    j = sitk.ReadImage(os.path.join(imgpath,imgfile[num]))
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(i)
    resampled = resampler.Execute(j)
    resampfile = os.path.basename(imgfile[num])
    
    sitk.WriteImage(resampled, os.path.join(resampath,resampfile))
    
    if num < num_img-1:
        num = num + 1

In [10]:
# resample mask 

resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_roi_niigz/'
# resampath  = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_org_niigz/'
resampfile = os.listdir(resampath)

i = sitk.ReadImage(os.path.join(maskpath,maskfile[0]))
sitk.WriteImage(i, os.path.join(resampath,maskfile[0]))
num_img = len(imgfile)

num = 1

for item_seg in os.listdir(maskpath):
   
    j = sitk.ReadImage(os.path.join(maskpath,maskfile[num]))
    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(i)
    resampled = resampler.Execute(j)
    resampfile = os.path.basename(maskfile[num])
    
    sitk.WriteImage(resampled, os.path.join(resampath,resampfile))
    
    if num < num_img-1:
        num = num + 1

In [ ]:
# feature for deep learning (2D): multiple slices (4 MM types)

resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_niigz/'
resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_org_niigz/'
resampfile = os.listdir(resampath)
num_seg = len(resampfile)
numslice = 5
target = np.zeros(num_seg*numslice)

first_num = 230 
first = sitk.ReadImage(os.path.join(resampath,resampfile[first_num]))
size = first.GetSize()
mid = size[2]/2
mid = math.ceil(mid)

feature = sitk.GetArrayFromImage(first)[mid-2:mid+3,:,:] # 5 slices per subject 
feature_reshape = feature.reshape(feature.shape[1],feature.shape[2],feature.shape[0])

resampfile = os.listdir(resampath)
num = first_num + 1

for item_mask in os.listdir(resampath):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[num]))
    j = sitk.GetArrayFromImage(i)[math.ceil(mid)-2:math.ceil(mid)+3,:,:]
    j_reshape = j.reshape(feature.shape[1],feature.shape[2],feature.shape[0])
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    resfile = os.path.basename(resampfile[num])
    
    if resfile[3] == '1':
        target[num*numslice:(num+1)*numslice] = 1
    else if resfile[3] == '2':
        target[num*numslice:(num+1)*numslice] = 2
        else if resfile[3] == '3':
            target[num*numslice:(num+1)*numslice] = 3
            else:
                target[num*numslice:(num+1)*numslice] = 4 
    
    if num < 454-1:
        num = num + 1

In [ ]:
# feature for deep learning (2D): multiple slices (3 ISS types)

resampath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_niigz/'
resampath = '/Users/hwang/Documents/research/1_MM/Data/1_ISS/resample_org_niigz/'

resampfile = os.listdir(resampath)
num_img = len(resampfile)
numslice = 5
target = np.zeros(num_seg*numslice)

first_num = 0 
first = sitk.ReadImage(os.path.join(resampath,resampfile[first_num]))
size = first.GetSize()
mid = size[2]/2
mid = math.ceil(mid)

feature = sitk.GetArrayFromImage(first)[mid-2:mid+3,:,:] # 5 slices per subject 
feature_reshape = feature.reshape(feature.shape[1],feature.shape[2],feature.shape[0])

resampfile = os.listdir(resampath)
num = first_num + 1

for item_mask in os.listdir(resampath):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[num]))
    j = sitk.GetArrayFromImage(i)[math.ceil(mid)-2:math.ceil(mid)+3,:,:]
    j_reshape = j.reshape(feature.shape[1],feature.shape[2],feature.shape[0])
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    resfile = os.path.basename(resampfile[num])
    
    if resfile[4] == '1':
        target[num*numslice:(num+1)*numslice] = 1
    else if resfile[4] == '2':
        target[num*numslice:(num+1)*numslice] = 2
            else:
                target[num*numslice:(num+1)*numslice] = 3 
    
    if num < num_img-1:
        num = num + 1

In [3]:
mid = 3
range(mid-2,mid+3)

range(1, 6)

In [ ]:
# feature for deep learning (2D): multiple slices 

resampath = '/Users/hwang/Documents/research/1_MM/Data/2_benign_MM/resample_niigz/'
resampfile = os.listdir(resampath)
num_seg = len(resampfile)
numslice = 5
ran = numslice/2
ran = math.floor(ran)
target = np.zeros(num_seg*numslice)

first = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
size = first.GetSize()
mid = size[2]/2
mid = math.ceil(mid)
feature = sitk.GetArrayFromImage(first)[mid-ran,:,:] # 5 slices per subject
feature_reshape = feature.transpose(1,2,0)

for n in range(mid-ran:mid+ran+1):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
    j = sitk.GetArrayFromImage(i)[n,:,:]
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    resfile = os.path.basename(resample[0])
    
    if resfile[2] == 'M':
        target[num*numslice:(num+1)*numslice] = 1
    else:
        target[num*numslice:(num+1)*numslice] = 0
        
num = 1

for n in range(mid-ran:mid+ran+1):

    i = sitk.ReadImage(os.path.join(resampath,resampfile[num]))
    j = sitk.GetArrayFromImage(i)[n,:,:]
    j_reshape = j.transpose(1,2,0)
    feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    resfile = os.path.basename(resampfile[num])
    
    for n in range(mid-ran:mid+ran+1):

        i = sitk.ReadImage(os.path.join(resampath,resampfile[0]))
        j = sitk.GetArrayFromImage(i)[n,:,:]
        j_reshape = j.transpose(1,2,0)
        feature_reshape = np.dstack((feature_reshape, j_reshape))
    
    
    if resfile[2] == 'M':
        target[num*numslice:(num+1)*numslice] = 1
    else:
        target[num*numslice:(num+1)*numslice] = 0
    
    if resfile[2] == 'M':
        target[num*numslice:(num+1)*numslice] = 1
    else:
        target[num*numslice:(num+1)*numslice] = 0
    
    if num < num_seg-1:
        num = num + 1